# Parte inicial del ETL creado para coderhouse

## Importamos las librerias necesarias

In [1]:
import logging
import json
import concurrent.futures
from redshift_connection import RedshiftConnection
from extract import ExtractData

Creamos el log para tener de manera visual si todo funciona

In [2]:
logging.basicConfig(
    filename='app.log',  # Nombre y ubicación del archivo de registro
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

In [3]:
config_file_path = 'redshift_config.ini'
redshift_conn = RedshiftConnection(config_file_path)
extract = ExtractData()

In [4]:
def redshift_charge(df_movie_order,df_overview,df_movie_genres):

    redshift_conn.connect()

    redshift_conn.insert('movies', df_movie_order)
    redshift_conn.insert('overviews', df_overview)
    redshift_conn.insert('movie_genres', df_movie_genres)

    redshift_conn.close()


In [5]:
nombre_archivo = "added_items.json"

lista_de_ids = []

# Leer el archivo JSONL línea por línea
with open(nombre_archivo, "r") as archivo:
    for linea in archivo:
        # Cargar el objeto JSON de la línea actual
        objeto_json = json.loads(linea)
        
        # Obtener el valor de "id" y agregarlo a la lista
        id_valor = objeto_json.get("id")
        if id_valor is not None:
            lista_de_ids.append(id_valor)



In [6]:
# Número de hilos o procesos en paralelo (ajusta según sea necesario)
num_hilos = 10

# Crea un ThreadPoolExecutor para manejar los hilos en paralelo
with concurrent.futures.ThreadPoolExecutor(max_workers=num_hilos) as executor:
    # Envía los elementos en paralelo
    resultados = list(executor.map(extract.extract, lista_de_ids))

    redshift_charge(resultados[0],resultados[1],resultados[2])

# resultados contiene los resultados de las operaciones de envío (puedes manejarlos según sea necesario)

KeyError: "['id'] not in index"